# ANALYZE HEART ACTIVITY: BOUTS AND FREQUENCY

# IMPORT all the modules you will need

In [1]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
from scipy import signal
import pandas as pd
import os
import matplotlib.colors as colors
from scipy.signal import argrelextrema

import operator

import seaborn as sns; sns.set()

import math
import matplotlib.patches as patches

from pandas import DataFrame
from scipy.interpolate import interp1d

from scipy.signal import decimate

from operator import itemgetter
from itertools import groupby


from itertools import chain
from scipy import stats

from scipy.stats import ks_2samp
from scipy.stats.stats import pearsonr 

from statsmodels.sandbox.stats.multicomp import multipletests

# Define path and frequence 

In [2]:

day = 'June 18', 'Oct 18', 'Nov 18'

path ='/Users/Natalia/Documents/DATA/Analysis w- tinDs 1do/'


fq= 29.98   #frames per second

f=60.03 # frames per second interpolated signal

factor = 2.0023   # to get the interpolated position of loomings

reso=300  # PLots resolution to save

# Def position of the fake loomings during baseline and stimuli

In [3]:
#times = [300, 283, 269, 250, 231, 213, 198, 186, 169, 155, 140, 126, 112, 101,  89,  71, 57,  43,  26,  13]
#sum_sec= [0,  17,  31,  50,  69,  87,  102, 114, 131, 145, 160, 174, 188, 199, 211, 229, 243, 257, 274, 287]
#sec  =[0,  17,   14,  19,  19,  18,  15,  12,  17,   14,  15,  14, 14,  11,  12,  18,  14,  14,  17,  13]
    
fake_sec= [2,  17,  31,  50,  69,  87,  102, 114, 131, 145, 160, 174, 188, 199, 211, 229, 243, 257, 274, 287]
fake_ini =[]
for fi in fake_sec:
    fi= fi*f
    fake_ini.append(int(round(fi)))

fake_end =[]
for fj in fake_sec:
    fj= (fj+0.5)*f
    fake_end.append(int(round(fj)))
fake_position = pd.DataFrame(data={'f_ending': fake_end, 'f_starting': fake_ini})


  
loom_sec= [300,  317,  331,  350,  369,  387,  402, 414, 431, 445, 460, 474, 488, 499, 511, 529, 543, 557, 574, 587]
ini =[]
for i in loom_sec:
    i= i*f
    ini.append(int(round(i)))

end =[]
for j in loom_sec:
    j= (j+0.5)*f
    end.append(int(round(j)))
position = pd.DataFrame(data={'ending': end, 'starting': ini})
#position


ini_fake =[]
for i in loom_sec:
    i= (i+7)*f
    ini_fake.append(int(round(i)))

end_fake =[]
for j in loom_sec:
    j= (j+7.5)*f
    end_fake.append(int(round(j)))
position_fake = pd.DataFrame(data={'ending': end_fake, 'starting': ini_fake})
#position_fake

  

# Read velocity, movement and cut them in 601 s

In [4]:
Light = [] 
for item in os.listdir(path):
    if item.startswith('Stimulus'):
        light = pd.read_csv(item,' ', header = None)
        frames_light= light.loc[light[1] > 5000, 0]
        frames_light=frames_light.tolist()
        #frames_light= light.pop(1)
        Light.append(frames_light)

print len(Light)


116


In [5]:
# Read Velocity
x=0
Velocity = [] 
for item in os.listdir(path):
    if item.startswith('BallActivity'):
        ball_act = pd.read_csv(item,' ', header = None)
        ball_extra = ball_act.pop(3)        
        ball_act= pd.DataFrame({'X':ball_act[0],'Y':ball_act[1],'Z':ball_act[2]*0.244}) # THis value is to fix an error in bonsai analysis
        velocity = np.sqrt(ball_act['X']**2 + ball_act['Y']**2 + ball_act['Z']**2)
        vel_aver= velocity.rolling(5, center=True).mean()                        # I'm doing a rolling mean!!!!!
        a= Light[x]                             #Cut the files acording to the first frame with LED light
        b= vel_aver[a[0]:a[0]+36078].tolist()
        Velocity.append(b)
        x=x+1
        
print len(Velocity[0])
print type(Velocity[0])

36078
<type 'list'>


In [6]:
#Read leg movements
x=0
Leg_mov= [] 
for item in os.listdir(path):
    if item.startswith('FlyActivity'):
        fly_act = pd.read_csv(item,' ', header = None) 
        leg_act= fly_act.drop([0,2,3], axis=1)      
        leg_aver= leg_act.rolling(5, center=True).mean()    # I'm doing a rolling mean!!!!!
        leg_a=leg_aver.values.tolist() 
        flattened = []
        for sublist in leg_a:
            for val in sublist:
                flattened.append(val/1)
        a= Light[x]
        b= flattened[a[0]:a[0]+36078]                         #Cut the files acording to the first frame with LED light 
        Leg_mov.append(b)
        x=x+1
print type(Leg_mov[0][0])

<type 'float'>


# READ THE X AXIS

two cells

In [7]:
#Read X files, fill the nan and create DataFrames   for LOOM
x_files = []

for item in os.listdir(path):
    if item.startswith('X_Y_mov'):
        df_x = pd.read_csv(item,' ', header = None)
        df_xa = df_x.pop(0)    # remove one columm extra 
        df_xp = df_x.pop(2)
        df_xa = np.array(df_xa)
        oka = ~np.isnan(df_xa)
        xpa = oka.ravel().nonzero()[0]
        fpa = df_xa[~np.isnan(df_xa)]
        xa  = np.isnan(df_xa).ravel().nonzero()[0]
        df_xa[np.isnan(df_xa)] = np.interp(xa, xpa, fpa)   
        df_xa -= np.nanmean(df_xa); df_xa /= np.nanstd(df_xa)   # regularize datasets by subtracting mean and dividing by s.d.    
        x_files.append(df_xa)
        
        df_xp = np.array(df_xp)
        okp = ~np.isnan(df_xp)
        xpp = okp.ravel().nonzero()[0]
        fpp = df_xp[~np.isnan(df_xp)]
        xp  = np.isnan(df_xp).ravel().nonzero()[0]
        df_xp[np.isnan(df_xp)] = np.interp(xp, xpp, fpp)
        df_xp -= np.nanmean(df_xp); df_xp /= np.nanstd(df_xp)   # regularize datasets by subtracting mean and dividing by s.d. 
        x_files.append(df_xp)
        #print df_xp
print len(x_files)

232


In [8]:
print len(x_files[0])

18018


# Interpolate X signal

In [8]:
def interpol(wave):
    frames = np.arange(0,len(wave))

    y_interp = interp1d(frames, wave, kind='cubic', bounds_error=False) 

    new_frames = np.arange(0, len(wave), len(wave)/36078)
    #print len(y_interp(new_frames))
    return y_interp(new_frames)



In [9]:
Heart=[]
for ani in x_files:
    #heart= ani[3:-3]
    heart= interpol(ani)
    Heart.append(heart)

print len(Heart)


232


# IDENTIFICATION of BACKWARD AND FOREWARD BEATINGS

In [11]:
def peaks_less(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
        
    
    peaksmax_A = np.array(argrelextrema(waveA, np.less_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.less_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    #print peaksM_A
    #print peaksM_P
    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A #=len(waveA)  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     

    #print len(PeaksAList)

    PeaksPList = [0] * P #=len(waveP)  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      
    #PeaksA = pd.DataFrame(PeaksAList)
    #PeaksP = pd.DataFrame(PeaksPList)

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
 
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    
    #print Listpeaks
    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)

    
    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':# and Listpeaks[i-1] == None and Listpeaks[i-2] == None and Listpeaks[i-3] == None and Listpeaks[i-4] == None:       
                list_backward.append(i)
#                pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
        
                                
    print 'num of backward beatings=', len(list_backward)                                                                 
    #print 'num of foreward beatings=',len(list_foreward)
    return list_backward, list_foreward#,Listpeaks




In [12]:
def peaks_greater(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
           
    peaksmax_A = np.array(argrelextrema(waveA, np.greater_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.greater_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    #print peaksM_A
    #print peaksM_P
    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A #=len(waveA)  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     

    #print len(PeaksAList)

    PeaksPList = [0] * P #=len(waveP)  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      
    #PeaksA = pd.DataFrame(PeaksAList)
    #PeaksP = pd.DataFrame(PeaksPList)

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    
    #print Listpeaks
    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)


    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':# and Listpeaks[i-1] == None and Listpeaks[i-2] == None and Listpeaks[i-3] == None and Listpeaks[i-4] == None:       
                list_backward.append(i)
                #pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:# and Listpeaks[i-4]!= 'post':
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
                                
    print 'num of backward beatings=', len(list_backward)                                                                 
    #print 'num of foreward beatings=',len(list_foreward)
    return list_backward, list_foreward#,Listpeaks


In [13]:
# Separate in different arrays each backward pulse and each foreward pulse

def direction(list_1, list_2):
    "Separate in different arrays each backward pulse and each foreward pulse"
    "these lists only contain indexes for the peaks"
    portions= np.split(list_1,np.searchsorted(list_1, list_2))
   
    return portions


In [14]:
# Get sequences only of the baseline or stimulation for heart data
Heart_total=[]
for ani in Heart:
    base= ani[0:]
    Heart_total.append(base)
print len(Heart_total)

Heart_base=[]
for ani in Heart:
    base= ani[0:18010]
    Heart_base.append(base)
    #print len(base)
    
Heart_stim=[]
for ani in Heart:
    stim= ani[18010:]
    Heart_stim.append(stim)

#print len(base)

    

232


In [16]:
# Run the function peaks for each animal and got peaks according to direction in anterior cell

#During whole experiment
backward_pulses_t=[]
#Listpeaks1=[]
#Listvalleys1=[]
for ani in range(0,len(Heart_total),2):  # Identify high and lower peaks and add them
    print ani//2
    back1, fore1, = peaks_greater(Heart_total[ani],Heart_total[ani+1],A=36078,P=36078)
    back2, fore2= peaks_less(Heart_total[ani],Heart_total[ani+1],A=36078,P=36078)   #, valleys1
    #Listpeaks1.append(peaks1)
    #Listvalleys1.append(valleys1)
    fore3=fore1+fore2
    back3=back1+back2
    w= sorted(fore3)
    y= sorted(back3)

    backward=direction(y,w)
    back_clean=[]
    for each in backward:
        if len(each)>2: 
            for e in each:
                back_clean.append(e)
    backward_pulses_t.append(back_clean) 

        

0
num of peaks in A= 3416 num of peaks in P= 3444
num of backward beatings= 2515
num of peaks in A= 3301 num of peaks in P= 3383
num of backward beatings= 2420
1
num of peaks in A= 3513 num of peaks in P= 3561
num of backward beatings= 2497
num of peaks in A= 3540 num of peaks in P= 3591
num of backward beatings= 2277
2
num of peaks in A= 3536 num of peaks in P= 3405
num of backward beatings= 2719
num of peaks in A= 3510 num of peaks in P= 3419
num of backward beatings= 2600
3
num of peaks in A= 3460 num of peaks in P= 3468
num of backward beatings= 3177
num of peaks in A= 3448 num of peaks in P= 3469
num of backward beatings= 3374
4
num of peaks in A= 3738 num of peaks in P= 3680
num of backward beatings= 2344
num of peaks in A= 3718 num of peaks in P= 3757
num of backward beatings= 2225
5
num of peaks in A= 3611 num of peaks in P= 3673
num of backward beatings= 2483
num of peaks in A= 3536 num of peaks in P= 3617
num of backward beatings= 2666
6
num of peaks in A= 3856 num of peaks i

num of backward beatings= 2503
num of peaks in A= 2924 num of peaks in P= 2944
num of backward beatings= 1796
53
num of peaks in A= 3667 num of peaks in P= 3616
num of backward beatings= 2112
num of peaks in A= 3693 num of peaks in P= 3657
num of backward beatings= 2505
54
num of peaks in A= 3261 num of peaks in P= 3369
num of backward beatings= 2230
num of peaks in A= 3270 num of peaks in P= 3349
num of backward beatings= 2070
55
num of peaks in A= 3558 num of peaks in P= 3551
num of backward beatings= 2503
num of peaks in A= 3549 num of peaks in P= 3544
num of backward beatings= 2377
56
num of peaks in A= 3629 num of peaks in P= 3712
num of backward beatings= 2761
num of peaks in A= 3734 num of peaks in P= 3726
num of backward beatings= 2821
57
num of peaks in A= 3249 num of peaks in P= 3247
num of backward beatings= 2237
num of peaks in A= 3229 num of peaks in P= 3228
num of backward beatings= 2203
58
num of peaks in A= 3407 num of peaks in P= 3310
num of backward beatings= 2043
num

num of backward beatings= 3430
105
num of peaks in A= 3930 num of peaks in P= 3437
num of backward beatings= 1145
num of peaks in A= 3921 num of peaks in P= 3515
num of backward beatings= 1399
106
num of peaks in A= 3852 num of peaks in P= 3632
num of backward beatings= 3189
num of peaks in A= 3880 num of peaks in P= 3734
num of backward beatings= 3431
107
num of peaks in A= 3904 num of peaks in P= 3806
num of backward beatings= 3283
num of peaks in A= 3923 num of peaks in P= 3740
num of backward beatings= 3383
108
num of peaks in A= 3937 num of peaks in P= 3937
num of backward beatings= 3161
num of peaks in A= 3949 num of peaks in P= 3902
num of backward beatings= 2915
109
num of peaks in A= 3834 num of peaks in P= 3853
num of backward beatings= 2700
num of peaks in A= 3881 num of peaks in P= 3866
num of backward beatings= 2746
110
num of peaks in A= 3547 num of peaks in P= 3495
num of backward beatings= 1914
num of peaks in A= 3571 num of peaks in P= 3500
num of backward beatings= 17

In [19]:
# Remove those peaks that are too close from the previous one


for ani in backward_pulses_t:
    bck_deleted=[]
    for i in range(0,1):
        if ani[i+1]< ani[i]+3:       
            bck_deleted.append(i)

    for i in range(1, len(ani)-1):
        if ani[i+1]< ani[i]+3:# or ani[i-1]> ani[i]-3:       
            bck_deleted.append(i)

    for i in range(len(ani)-1, len(ani)):
        if ani[i]< ani[i-1]+3:       
            bck_deleted.append(i)
    
    #print bck_deleted
    for index in sorted(bck_deleted, reverse=True):
        del ani[index]


In [21]:
# Slipt the peaks in fragments according to the distance among them

t_periods_backward=[]
for ani in backward_pulses_t:
    back_frag=[]
    array=[]
    array1=[]

    for i in range(0, len(ani)-1):
        if ani[i]+18 >= ani[i+1]:
            array.append(ani[i])
    
        else:
            array.append(ani[i])
            back_frag.append(array)
            array=[]
            #if ani[i] >= ani[i+1]+20:
             #   array.append(ani[i])
        
    back_per=[]       
    for x in back_frag:             # Remove those fragments with less tham 3 peaks.
        if len(x)>3:
            back_per.append(x)
    t_periods_backward.append(back_per)       
print len(t_periods_backward)


116


In [24]:
def peaks_less1(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase but excluding anpo peaks from backward list"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
        
    
    peaksmax_A = np.array(argrelextrema(waveA, np.less_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.less_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A #=len(waveA)  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     


    PeaksPList = [0] * P #=len(waveP)  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
 
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    
    #print Listpeaks
    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)

    
    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':# and Listpeaks[i-1] == None and Listpeaks[i-2] == None and Listpeaks[i-3] == None and Listpeaks[i-4] == None:       
                #list_foreward.append(i)
                pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
        
                                
    print 'num of backward beatings=', len(list_backward)                                                                 

    return list_backward, list_foreward#,Listpeaks



In [25]:
def peaks_greater1(waveA,waveP, A=36078,P=36078):
    "Identify the peaks when waves are in same phase but excluding anpo peaks from backward list"
    "And separate them in two list:indexes for retrograde beating and anterograde beatings "
           
    peaksmax_A = np.array(argrelextrema(waveA, np.greater_equal,order=6))
    peaksmax_P = np.array(argrelextrema(waveP, np.greater_equal,order=6))

    peaksM_A= peaksmax_A[0] # In other way I have an array of array
    peaksM_P= peaksmax_P[0]

    print 'num of peaks in A=', len(peaksM_A), 'num of peaks in P=', len(peaksM_P)

    PeaksAList = [0] * A #=len(waveA)  # Create list of 18054 '0's 
    for i in peaksM_A:   #add the index of peaks
        PeaksAList[i] = i     

    #print len(PeaksAList)

    PeaksPList = [0] * P #=len(waveP)  # Create list of 18054 '0's
    for j in peaksM_P:       #add the index of peaks
        PeaksPList[j] = j      

    Listpeaks = []    #Creates a list where in each frame says if peaks happens in anterior 'ant'  posterior 'post or both 'anpo
    for i in range(0, len(PeaksPList)):
        if PeaksAList[i] == 0 == PeaksPList[i]:
            Listpeaks.append(None)     
        else:
            if PeaksAList[i] > PeaksPList[i]:
                Listpeaks.append('ant')
            else:
                if PeaksAList[i] == PeaksPList[i]:
                    Listpeaks.append('anpo')
                else:
                    if PeaksAList[i] < PeaksPList[i]:
                        Listpeaks.append('post') 
    
    #print Listpeaks
    list_backward= []  #list of index of peaks coresponding only to retrograde beating (anterior cell beating first)
    list_foreward= []   #list of index of peaks coresponding only to anterograde beating (posterior cell beating first)


    for i in range(0, len(Listpeaks)-12):
        if Listpeaks[i] == None:
            pass
        
        else:    
            if Listpeaks[i] == 'anpo':# and Listpeaks[i-1] == None and Listpeaks[i-2] == None and Listpeaks[i-3] == None and Listpeaks[i-4] == None:       
                #list_foreward.append(i)
                pass
              
            elif Listpeaks[i] == 'post' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None:# and Listpeaks[i-4]!= 'post':
                for b in range(1,7):
                    if Listpeaks[i+b] == 'ant':                 
                        list_foreward.append(i)
                    else:
                        pass
    
            elif Listpeaks[i] == 'ant' and Listpeaks[i-1]== None and Listpeaks[i-2]== None and Listpeaks[i-3]== None and Listpeaks[i-4]== None: 
                for b in range(1,7):
                    if Listpeaks[i+b] == 'post':
                        list_backward.append(i)
                    else:
                        pass
                                
    print 'num of backward beatings=', len(list_backward)                                                                 
    #print 'num of foreward beatings=',len(list_foreward)
    return list_backward, list_foreward#,Listpeaks


In [26]:
# Select animals with cells in opposite walls for analysis 
# second list to analyze with peaks_1
del_beh=[3,6,9,10,12,17,18,19,20,21,22,26,27,28,29,30,31,32,33,34,36,37,38,39,43,46,47,48,49,50,52,59,60,61,64,66,69,70,71,72,74,75,76,77,78,79,81,82,84,87,88,89,90,91,94,96,97,98,101,102,103,104,105,106,107,108,109,112,113,114,115]

#animals to remove from the Heart_total list
del_beh_b=[3,6,9,10,12,17,18,19,20,21,22,26,27,29,30,31,32,33,36,37,38,39,43,46,47,48,49,50,52,59,60,61,64,66,69,70,71,72,74,75,76,77,78,79,81,82,84,87,88,89,90,91,94,96,97,98,101,102,103,104,105,106,107,108,109,112,113,114,115]

Heart_total_2=[]
Heart_base_2=[]
Heart_stim_2=[]
for ani in del_beh:
    Heart_base_2.append(Heart_base[ani*2])
    Heart_base_2.append(Heart_base[(ani*2)+1])
    Heart_stim_2.append(Heart_stim[ani*2])
    Heart_stim_2.append(Heart_stim[(ani*2)+1])
    Heart_total_2.append(Heart_total[ani*2])
    Heart_total_2.append(Heart_total[(ani*2)+1])
    

In [27]:
# Run the function peaks for each  animal and got peaks according to direction

#During whole experiment
backward_pulses_t2=[]
#Listpeaks2=[]
#Listvalleys2=[]
for ani in range(0,len(Heart_total_2),2):  # Identify high and lower peaks and add them
    print ani//2
    back1, fore1= peaks_greater1(Heart_total_2[ani],Heart_total_2[ani+1],A=36078,P=36078)#,peaks2
    back2, fore2= peaks_less1(Heart_total_2[ani],Heart_total_2[ani+1],A=36078,P=36078)   #,valleys2
    #Listpeaks2.append(peaks2)
    #Listvalleys2.append(valleys2)
    fore3=fore1+fore2
    back3=back1+back2
    w= sorted(fore3)
    y= sorted(back3)
    
    #foreward = direction(w, y)
    backward=direction(y,w)
    back_clean=[]
    for each in backward:
        if len(each)>2: 
            for e in each:
                back_clean.append(e)
    backward_pulses_t2.append(back_clean) 

0
num of peaks in A= 3460 num of peaks in P= 3468
num of backward beatings= 2078
num of peaks in A= 3448 num of peaks in P= 3469
num of backward beatings= 2404
1
num of peaks in A= 3856 num of peaks in P= 3318
num of backward beatings= 2267
num of peaks in A= 3800 num of peaks in P= 3352
num of backward beatings= 2116
2
num of peaks in A= 3529 num of peaks in P= 3458
num of backward beatings= 2227
num of peaks in A= 3540 num of peaks in P= 3483
num of backward beatings= 2255
3
num of peaks in A= 3827 num of peaks in P= 3812
num of backward beatings= 2209
num of peaks in A= 3828 num of peaks in P= 3856
num of backward beatings= 2346
4
num of peaks in A= 3455 num of peaks in P= 3451
num of backward beatings= 2064
num of peaks in A= 3508 num of peaks in P= 3510
num of backward beatings= 1582
5
num of peaks in A= 3577 num of peaks in P= 3371
num of backward beatings= 2340
num of peaks in A= 3514 num of peaks in P= 3184
num of backward beatings= 2142
6
num of peaks in A= 3716 num of peaks i

num of backward beatings= 1531
num of peaks in A= 3649 num of peaks in P= 3858
num of backward beatings= 2077
52
num of peaks in A= 4003 num of peaks in P= 3903
num of backward beatings= 1711
num of peaks in A= 4042 num of peaks in P= 3958
num of backward beatings= 2123
53
num of peaks in A= 3178 num of peaks in P= 2918
num of backward beatings= 1871
num of peaks in A= 3156 num of peaks in P= 2945
num of backward beatings= 1864
54
num of peaks in A= 3633 num of peaks in P= 3297
num of backward beatings= 2235
num of peaks in A= 3632 num of peaks in P= 3307
num of backward beatings= 2060
55
num of peaks in A= 3512 num of peaks in P= 3468
num of backward beatings= 2417
num of peaks in A= 3549 num of peaks in P= 3481
num of backward beatings= 2305
56
num of peaks in A= 3509 num of peaks in P= 3613
num of backward beatings= 1373
num of peaks in A= 3389 num of peaks in P= 3568
num of backward beatings= 703
57
num of peaks in A= 3439 num of peaks in P= 3412
num of backward beatings= 1887
num 

In [29]:
# Remove those peaks that are too close from the previous one


for ani in backward_pulses_t2:
    bck_deleted=[]
    for i in range(0,1):
        if ani[i+1]< ani[i]+3:       
            bck_deleted.append(i)

    for i in range(1, len(ani)-1):
        if ani[i+1]< ani[i]+3:# or ani[i-1]> ani[i]-3:       
            bck_deleted.append(i)

    for i in range(len(ani)-1, len(ani)):
        if ani[i]< ani[i-1]+3:       
            bck_deleted.append(i)
    
    #print bck_deleted
    for index in sorted(bck_deleted, reverse=True):
        del ani[index]

In [31]:
# Split the peaks in fragments according to the distance among them

t_periods_backward_2=[]
for ani in backward_pulses_t2:
    back_frag=[]
    array=[]
    array1=[]

    for i in range(0, len(ani)-1):
        if ani[i]+18 >= ani[i+1]:
            array.append(ani[i])
    
        else:
            array.append(ani[i])
            back_frag.append(array)
            array=[]
            #if ani[i] >= ani[i+1]+20:
             #   array.append(ani[i])
        
    back_per=[]       
    for x in back_frag:             # Remove those fragments with less tham 3 peaks.
        if len(x)>3:
            back_per.append(x)
    t_periods_backward_2.append(back_per)       
print len(t_periods_backward_2)


71


In [33]:


#define files heart to discard from the total list

del_hearts_b=[]
for each in del_beh_b: # this is the clean list of heart 
    del_hearts_b.append(each*2)
    del_hearts_b.append((each*2)+1)     
print del_hearts_b


[6, 7, 12, 13, 18, 19, 20, 21, 24, 25, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 52, 53, 54, 55, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 72, 73, 74, 75, 76, 77, 78, 79, 86, 87, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 118, 119, 120, 121, 122, 123, 128, 129, 132, 133, 138, 139, 140, 141, 142, 143, 144, 145, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 163, 164, 165, 168, 169, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 188, 189, 192, 193, 194, 195, 196, 197, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 224, 225, 226, 227, 228, 229, 230, 231]


In [34]:
# remove animlas for the second list
Velocity_2=[]
Light_2=[]
Leg_mov_2=[]
for ani in del_beh:
    
    Velocity_2.append(Velocity[ani])
    Light_2.append(Light[ani])
    Leg_mov_2.append(Leg_mov[ani])

In [35]:
#define files heart_2 to discard 

del_beh_2=[0,12,13,14,15,16,17,19,20,22,23,24,30,32,37,38,41,43,48,49,51,53,54,56,58,61,62,67]
del_hearts_2=[]
for each in del_beh_2:
    del_hearts_2.append(each*2)
    del_hearts_2.append((each*2)+1)

print del_hearts_2

[0, 1, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 60, 61, 64, 65, 74, 75, 76, 77, 82, 83, 86, 87, 96, 97, 98, 99, 102, 103, 106, 107, 108, 109, 112, 113, 116, 117, 122, 123, 124, 125, 134, 135]


In [36]:
for index in sorted(del_hearts_b, reverse=True):
    del Heart_base[index]
    del Heart_stim[index]
    del Heart_total[index]
    
print len(Heart_total)

94


In [37]:
for index in sorted(del_hearts_2, reverse=True):
    del Heart_base_2[index]
    del Heart_stim_2[index]
    del Heart_total_2[index]
    
print len(Heart_total_2)

86


In [38]:
for index in sorted(del_beh_b, reverse=True):
    del Velocity[index]
    del Leg_mov[index]
    del Light[index]
    del t_periods_backward[index]
    del b_periods_backward[index]
    del s_periods_backward[index]

print len(Velocity)
print len(Light)
print len(Leg_mov)
print len(t_periods_backward)

47
47
47
47


In [39]:
for index in sorted(del_beh_2, reverse=True):
    del Velocity_2[index]
    del Leg_mov_2[index]
    del Light_2[index]
    del t_periods_backward_2[index]
    del b_periods_backward_2[index]
    del s_periods_backward_2[index]

    
print len(Velocity_2)
print len(Light_2)
print len(Leg_mov_2)
print len(t_periods_backward_2)

43
43
43
43


In [40]:
for index in sorted(del_beh_b, reverse=True):

    del backward_pulses_t[index]
    del backward_pulses_b[index]
    del backward_pulses_s[index]

print len(backward_pulses_t)

47


In [41]:
for index in sorted(del_beh_2, reverse=True):

    del backward_pulses_t2[index]
    del backward_pulses_b2[index]
    del backward_pulses_s2[index]


print len(backward_pulses_t2)

43


In [42]:
# These final lists are ordered for dots stimulated animals and then loomed ones. However the indexes don't correspond anymore to thenumber of fly when recorded
Heart_base_new=Heart_base_2[0:22]+Heart_base+Heart_base_2[22:]
Heart_stim_new=Heart_stim_2[0:22]+Heart_stim+Heart_stim_2[22:]
Heart_total_new=Heart_total_2[:22]+Heart_total+Heart_total_2[22:]
Velocity_new=Velocity_2[0:11]+Velocity+Velocity_2[11:]
Leg_mov_new=Leg_mov_2[0:11]+Leg_mov+Leg_mov_2[11:]
Light_new=Light_2[0:11]+Light+Light_2[11:]
t_periods_backward_new=t_periods_backward_2[0:11]+t_periods_backward+t_periods_backward_2[11:]
b_periods_backward_new=b_periods_backward_2[0:11]+b_periods_backward+b_periods_backward_2[11:]
s_periods_backward_new=s_periods_backward_2[0:11]+s_periods_backward+s_periods_backward_2[11:]
backward_pulses_new=backward_pulses_t2[0:11]+backward_pulses_t+backward_pulses_t2[11:]
print len(Velocity_new)

90


# Get arrays of 10 and -10 to represent forward and backward periods 

In [45]:

## (10 FORWARD  and -10 BACKWARD)
ones_zeros_base=[]
for ani in b_periods_backward_new:
    ani0=[10]*18010
    for each in ani:
        for num in range(each[0],each[-1]):
            ani0[num]=-10
    ones_zeros_base.append(ani0)
ones_zeros_stim=[]
for ani in s_periods_backward_new:
    ani0=[10]*18058
    for each in ani:
        for num in range(each[0],each[-1]):
            ani0[num]=-10
    ones_zeros_stim.append(ani0)
    
zeros_total=[]
for ani in t_periods_backward_new:
    ani0=[10]*36068
    for each in ani:
        for num in range(each[0],each[-1]):
            ani0[num]=-10
    zeros_total.append(ani0)


# GET PERIODS WHEN FLIES ARE RUNNING GROOMING OR FREEZING

In [46]:
#Return periods of immobilization during baseline and stimulation according to ball movement

def moving_ball(list):
    "return the indexes of the periods longer than 2.5 sec where the fly is NOT walking during baseline and stimulation"
    nowalkbase=[]
    nowalkstim=[]
    for i, j in enumerate(list):
        if i<18010 and j < 0.5:#1.0:
            nowalkbase.append(i)
        else:
            if 36078>i>= 18010 and j<0.5:#1.0:
                nowalkstim.append(i)
                
    quietbase=[]
    for k, g in groupby(enumerate(nowalkbase), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietbase.append(stop)

    quietstim=[]
    for k, g in groupby(enumerate(nowalkstim), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietstim.append(stop)
    
    nowalkingbase=[]
    nowalkingstim=[]
    for array in quietbase:
        if len(array)>150:   # 2.5 seconds
            nowalkingbase.append(array)
        else:
            pass
    for array in quietstim:
        if len(array)>150:
            nowalkingstim.append(array)
        else:
            pass
    print len(nowalkingbase),len(nowalkingstim)
    return nowalkingbase, nowalkingstim

In [47]:
# Return freezing periods during baseline and stimulation according to legs movement
def freezing_legs(list):
    "return the indexes of the periods longer than 2.5 sec where the fly is immobile during baseline and stimulation"
    freezbase=[]
    freezstim=[]
    for i, j in enumerate(list):
        if i<18010 and np.mean(list[i-5:i+5]) < 5:
            freezbase.append(i)
        else:
            if 36078>i>= 18010 and np.mean(list[i-5:i+5]) < 5:
                freezstim.append(i)
    
    quietbase=[]
    for k, g in groupby(enumerate(freezbase), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietbase.append(stop)

    quietstim=[]
    for k, g in groupby(enumerate(freezstim), lambda (i,x):i-x):    # To make groups with the indexes  for each period
        stop= map(itemgetter(1), g)
        quietstim.append(stop)
    
    freezingbase=[]
    freezingstim=[]
    for array in quietbase:
        if len(array)>150:    # 2.5 seconds
            freezingbase.append(array)
        else:
            pass
    for array in quietstim:
        if len(array)>150:
            freezingstim.append(array)
        else:
            pass
    print len(freezingbase),len(freezingstim)
    return freezingbase, freezingstim

In [48]:
# Identify no walking periods in Velocity file  #GET LIST OF INDEXES 


NoWalkbase=[]     
NoWalkstim=[]

for ani in Velocity_new:
    nowalkingbase, nowalkingstim = moving_ball(ani)
    NoWalkbase.append(nowalkingbase)
    NoWalkstim.append(nowalkingstim)
    
print len(NoWalkbase)

7 8
14 21
2 2
7 1
10 14
2 0
9 3
19 16
7 8
14 11
4 1
2 4
7 5
11 10
5 7
15 9
11 7
2 3
1 2
11 13
22 17
5 7
8 3
4 4
2 0
13 10
11 2
14 4
15 10
4 9
4 9
3 8
14 7
8 2
6 22
18 7
8 8
14 4
7 15
13 1
12 1
8 6
8 2
17 1
10 13
5 7
25 7
3 4
26 13
28 12
2 2
4 10
4 2
29 4
7 20
18 1
4 16
1 1
14 15
15 10
4 6
11 10
10 5
6 8
13 1
18 21
13 6
14 9
7 13
8 2
13 11
23 20
13 3
9 10
14 1
7 14
2 5
6 12
12 16
20 5
19 13
20 11
12 17
12 3
2 8
14 2
11 19
16 18
21 8
15 3
90


In [50]:
# Identify moving periods in leg file  # GET LIST OF INDEXES 

Freezbase=[]
Freezstim=[]   

for ani in Leg_mov_new:
    freezingbase, freezingstim = freezing_legs(ani)
    Freezbase.append(freezingbase)
    Freezstim.append(freezingstim)
     
print len(Freezstim)

1 0
3 3
2 1
0 0
2 5
1 0
2 1
5 3
1 0
1 2
0 0
0 1
0 0
0 0
1 1
4 1
4 1
0 0
0 0
11 11
23 19
2 1
1 3
0 0
3 0
1 2
1 1
1 2
2 9
0 6
0 13
0 6
5 3
5 6
1 22
8 7
6 7
1 0
1 14
10 2
5 1
1 10
1 1
15 2
1 12
1 6
3 2
1 3
11 7
23 18
0 0
0 4
3 1
21 4
10 20
32 1
0 12
0 6
5 11
1 8
0 1
2 18
6 6
1 6
4 3
0 14
0 5
2 7
0 4
2 3
4 14
4 24
2 2
5 5
0 3
1 14
0 2
1 13
1 10
8 4
28 1
15 14
4 13
0 6
0 4
0 15
0 13
0 12
2 5
1 11
90


# separate freezers from runners according to time freezing during stimulation

In [51]:
# Time freezing

flat_Freezstim=[]
for ani in Freezstim[0:90]:
    b= list(chain.from_iterable(ani))
    flat_Freezstim.append(len(b))
    print len(b)

0
537
156
0
1125
0
191
562
0
369
0
200
0
0
198
170
218
0
0
2435
10111
261
492
0
0
381
161
18017
16973
1392
7412
1270
16752
14204
10468
16382
1529
0
6333
18019
18030
15259
17999
17813
4352
14711
18026
17729
2855
11441
0
11691
18060
17215
9140
18068
4323
18001
13939
14317
155
8961
17476
2136
17984
4141
9309
9646
1091
17216
6771
8059
17559
1004
17990
8940
317
2852
2806
17595
18068
13787
11080
15969
923
11731
9541
6162
14897
17658


In [52]:
freezers=[]
runners=[]
mixers=[]
for i, e in enumerate(flat_Freezstim[27:]):
    if e> 12600:
        freezers.append(i+27)
    elif e<5400:
        runners.append(i+27)
    else: 
        mixers.append(i+27)
print freezers
print runners
print mixers

[27, 28, 32, 33, 35, 39, 40, 41, 42, 43, 45, 46, 47, 52, 53, 55, 57, 58, 59, 62, 64, 69, 72, 74, 79, 80, 81, 83, 88, 89]
[29, 31, 36, 37, 44, 48, 50, 56, 60, 63, 65, 68, 73, 76, 77, 78, 84]
[30, 34, 38, 49, 51, 54, 61, 66, 67, 70, 71, 75, 82, 85, 86, 87]
